In [44]:
import time
import numpy as np
import pandas as pd
import jax.numpy as jnp
from src.mcnnm.util import generate_data
from src.mcnnm.estimate import estimate

In [45]:
import causaltensor.cauest.MCNNM as MC

# Comparison of Causaltensor, Fect, and Lightweight-MCNNM
This notebook compares the performance of Causaltensor, Fect, and Lightweight-MCNNM in estimating the average treatment effect in a panel data setting. The comparison is based on the following metrics: treatment effect estimate, execution time, and MSE of the untreated counterfactual outcome matrix. The comparison is based on a simulated dataset with 100 units and 100 periods without any covariates. The dataset is generated using the `generate_data` function from the `util` module. The true treatment effect is set to 1. The untreated counterfactual outcome matrix is also generated using the true parameters. The three estimators are then run on the generated data, and the results are compared. The reason why these estimators are compared without covariates is that they handle covariates differently: lightweight-mcnnm exactly follows the description in section 8.1 of [the paper](https://www.tandfonline.com/doi/full/10.1080/01621459.2021.1891924) and regularizes covariates separately, while causaltensor and fect do not handle covariates in the same way. Colab can not be used to run this notebook because it requires a local R installation. All results were obtained on a 10-core Apple M1 Pro CPU.

## Generate Data

In [46]:
nobs, nperiods = 100, 100
data, true_params = generate_data(nobs=nobs, nperiods=nperiods, seed=42, X_cov=False, Z_cov=False, V_cov=False,
                                  assignment_mechanism="block")

Y = jnp.array(data.pivot(index="unit", columns="period", values="y").values)
W = jnp.array(data.pivot(index="unit", columns="period", values="treat").values)

tau = true_params["treatment_effect"]
Y_0 = jnp.array(true_params["Y(0)"]).reshape(nobs, nperiods)

In [47]:
# Define a function to compute the MSE of two matrices
def mse(A, B):
    return jnp.mean((A - B)**2)

## Run all three estimators

### Causaltensor

In [48]:
# Code adapted from Causaltensor's Matrix Completion Example: https://colab.research.google.com/github/TianyiPeng/causaltensor/blob/main/tests/MCNNM_test.ipynb#scrollTo=LSYGyn4cl9Bd (last cell)
# Causaltensor nomenclature: observation matrix O and treatment pattern Z
# so O is Y and Z is W 
# Causaltensor by default uses 6 candidate lambdas
# input arrays have to be numpy
Y_np = np.array(data.pivot(index="unit", columns="period", values="y").values)
W_np = np.array(data.pivot(index="unit", columns="period", values="treat").values) 

causaltensor_start_time = time.time()
solver = MC.MCNNMPanelSolver(Z=W_np, O=Y_np)
ct_res = solver.solve_with_cross_validation(K=5)
causaltensor_exec_time = time.time() - causaltensor_start_time

/Users/ts/Library/Caches/pypoetry/virtualenvs/project-name-rtHbWh8M-py3.12/lib/python3.12/site-packages/causaltensor/cauest/MCNNM.py:251: RuntimeWarning: invalid value encountered in scalar divide
  res.tau = np.sum((self.O - res.baseline_model)*self.Z) / np.sum(self.Z)


### Lightweight-MCNNM

In [49]:
mcnnm_start_time = time.time()
results = estimate(Y, W, K=5, n_lambda_L=6)  # Causaltensor by default uses 6 candidate lambdas
mcnnm_exec_time = time.time() - mcnnm_start_time

### Fect

In [50]:
long_data = data.copy() # Create a long format DataFrame for fect
long_data = long_data.rename(columns={"unit": "id", "period": "time", "y": "Y"})  # Rename columns
long_data["D"] = W.flatten()  # Flatten the W matrix and add it as a new column
long_data = long_data.drop(columns=["treat"]) # drop column "treat"
long_data.to_csv("fect_data.csv", index=False)  # Save the long format DataFrame to a CSV file

After manually running the code contained in fect_test.R in Rstudio, fect version 0.1.0, we can load the results::

In [51]:
fect_results = pd.read_csv("fect_results.csv")  # Read the results

# Access the values
fect_tau = fect_results['att_avg'].values[0]
fect_lam = fect_results['lambda_cv'].values[0]
fect_Y_0 = fect_results.filter(regex='^Y_ct_').values
fect_exec_time = fect_results['elapsed_time'].values[0]

## Results Comparison:

In [52]:
print("Causaltensor:")
print(f"true effect: {tau}, estimated effect: {ct_res.tau:.4f}")
print(f"Execution time: {causaltensor_exec_time:.2f} s, Cross-validated lambda: Not made available by Causaltensor")
print("-"*100)
print("Fect:")
print(f"true effect: {tau}, estimated effect: {fect_tau:.4f}")
print(f"Execution time: {fect_exec_time:.2f} s, Cross-validated lambda: {fect_lam:.6f}")
print(f"MSE of Y(0) (The untreated counterfactual outcome matrix completed by these estimators ): {mse(Y_0, fect_Y_0):.4f}")
print("-"*100)
print("Lightweight-MCNNM:")
print(f"true effect: {tau}, estimated effect: {results.tau:.4f}")
print(f"Execution time: {mcnnm_exec_time:.2f} s, Cross-validated lambda: {results.lambda_L:.6f}")
print(f"MSE of Y(0) (The untreated counterfactual outcome matrix completed by these estimators ): {mse(Y_0, results.Y_completed):.4f}")

Causaltensor:
true effect: 1.0, estimated effect: 0.9394
Execution time: 36.82 s, Cross-validated lambda: Not made available by Causaltensor
----------------------------------------------------------------------------------------------------
Fect:
true effect: 1.0, estimated effect: 0.9639
Execution time: 13.46 s, Cross-validated lambda: 0.000061
MSE of Y(0) (The untreated counterfactual outcome matrix completed by these estimators ): 9.2464
----------------------------------------------------------------------------------------------------
Lightweight-MCNNM:
true effect: 1.0, estimated effect: 0.9777
Execution time: 0.91 s, Cross-validated lambda: 0.001000
MSE of Y(0) (The untreated counterfactual outcome matrix completed by these estimators ): 4.6382
